## Explore and cluster the neighborhoods in Toronto

1. Select the data which contains Toronto in their borough
2. Visualize the map using Folium
3. Cluster the neighborhoods based on their proximity
4. Visualize the map with clusters

In [1]:
# Import libraries

import pandas as pd
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
# In previous exercise i created a toronto.csv in order to get the final dataset after scraping and merging

df_tor = pd.read_csv('toronto.csv')
df_tor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


1. Select the data which contains Toronto in their borough

In [3]:
toronto = df_tor[df_tor['Borough'].str.contains('Toronto',regex=False)]
toronto.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
35,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


2. Visualize the map using Folium

In [4]:
map_tor = folium.Map(location=[43.651070,-79.347015],zoom_start=12)

for lat,lng,borough,neighbourhood in zip(toronto['Latitude'],toronto['Longitude'],toronto['Borough'],toronto['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_tor)
map_tor

3. Cluster the neighborhoods based in their proximity

In [5]:
k=5 # number of clusters

cluster_toronto = toronto.drop(['PostalCode','Borough','Neighborhood'],1)

K_means = KMeans(n_clusters = k,random_state=0) #instatiate the KMeans
K_means.fit(cluster_toronto) #Fit the model

K_means.labels_ #Labels of our model

array([4, 4, 4, 1, 4, 4, 0, 4, 3, 1, 4, 0, 1, 4, 0, 1, 4, 1, 2, 2, 2, 2,
       3, 2, 0, 3, 2, 0, 3, 2, 0, 2, 0, 4, 4, 4, 4, 4, 1])

In [6]:
toronto.insert(0, 'Cluster', K_means.labels_) #insert labels into a dataframe
toronto.head(10)

,Cluster,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,1,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,0,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,4,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,3,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
35,1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


4. Visualize the map with clusters

In [7]:
map_cluster = folium.Map(location=[43.651070,-79.347015],zoom_start=10) #Create map

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lon, neighborhood, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood'], toronto['Cluster']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster)
       
map_cluster